In [1]:
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_teddynote.messages import stream_response
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
import pandas
from llama_parse import LlamaParse
# from langchain_huggingface import HuggingFaceEmbeddings

In [3]:
# embed model

embed_model = HuggingFaceEmbeddings(
    model_name = "jhgan/ko-sroberta-nli",
    model_kwargs = {"device":"cpu"},
    encode_kwargs = {"normalize_embeddings":True}
)

C:\Users\jw\AppData\Local\Temp\ipykernel_20608\3248108824.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
c:\Users\jw\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:

llm = ChatOpenAI(
    temperature = 0,
    model_name = "gpt-4o-mini",
    max_tokens=500
)

In [5]:
# Load Split Embed function

def LSE(fname):
    
    # Load
    loader = TextLoader(fname, encoding = "utf-8")
    data = loader.load()
    splitter = CharacterTextSplitter(
    separator = "//<>",
    # chunk_size = size,
    # chunk_overlap = overlap,
    length_function = len
    )

    # Split
    documents = splitter.split_documents(data)
    
    #Embed
    vectorstore = FAISS.from_documents(documents,
                                   embedding = embed_model,
                                   distance_strategy = DistanceStrategy.COSINE)
    
    return vectorstore

In [6]:
#Prompt Chain Response function

def PCR(template, docs, query):
    
    # Prompt
    prompt = ChatPromptTemplate.from_template(template)

    # 이 부분은 뭔지 모르겠음
    # def format_docs(docs):
    #     return '\n\n'.join([d.page_content for d in docs])

    # Chain
    chain = prompt | llm | StrOutputParser()

    # Run
    response = chain.invoke({"context": docs, "question":query})
    return response


# 당신은 광운대학교 학생인 사용자에게 사용자가 입력한 자신의 입학 연도를 기반으로 졸업 요건을 알려주는 인공지능 챗봇입니다.
#             졸업 이수학점 표는 총 7열입니다. 이때 7열중 교양은 (필수+균형)과 기초라는 2열로 다시 나누어집니다.
#             주전공학점(필수 포함)은 단일 전공시와 다전공 이수시로 2열로 나누어집니다.
#             표에서 이웃한 여러 셀이 동일한 값을 가질 경우 그 열의 같은 값을 가지는 이웃한 셀들을 통합하기도 합니다.
#             또한 이 문서는 각 졸업 요건이 동일한 입학 연도 단위 앞 부분에 가., 나., 다. ... 순으로 번호를 매깁니다.
#             이 중 어떤 부분을 참고해서 답변 하였는지 맨 처음에 명시하세요.

In [7]:
# 공학인증 chain

# Engineering Chain
def EChain(docs, query):
    template1 = '''표의 내용에서 생략되는 정보 없이 모든 것을 출력해야 하며, 마크다운 형식으로 보기 편하게 출력하세요.
            오직 다음의 context에 기반하여 대답하세요. {context}, Question: {question}'''
    
    vectorstore1 = LSE(docs[0].page_content)
    retriever1 = vectorstore1.as_retriever(search_kwargs={'k': 1})
    docs1 = retriever1.get_relevant_documents(query)
    response1 = PCR(template1, docs1[0], query)

    vectorstore2 = LSE(docs[1].page_content)
    retriever2 = vectorstore2.as_retriever(search_kwargs={'k': 1})
    docs2 = retriever2.get_relevant_documents(query)
    response2 = PCR(template1, docs2, query)

    template2 = '''오직 context1과 context2만을 참고하여 두 내용을 보기 좋게 생략되는 정보 없이 마크다운 형식으로 출력하세요.
                {context1}, {context2}, Question: {question}'''
    prompt = ChatPromptTemplate.from_template(template2)
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"context1": response1, "context2": response2, "question":query})
    print(response)

In [8]:
# file list embedding

# Load
loader = TextLoader("file_list.txt", encoding = "utf-8")
data = loader.load()
splitter = CharacterTextSplitter(
separator = ",",
chunk_size = 20,
chunk_overlap = 0,
length_function = len
)

# Split
documents = splitter.split_documents(data)

#Embed
vectorstore = FAISS.from_documents(documents,
                                embedding = embed_model,
                                distance_strategy = DistanceStrategy.COSINE)

query = "2021학년도 소프트웨어학부 신입학자의 공학인증을 알려줘."

retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

docs = retriever.get_relevant_documents(query)

Created a chunk of size 25, which is longer than the specified 20
Created a chunk of size 25, which is longer than the specified 20
C:\Users\jw\AppData\Local\Temp\ipykernel_20608\233103367.py:25: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [9]:
print(docs[0].page_content)

if docs[0].page_content == "공학인증제도 안내 - 교양 및 교과과정표.md" or docs[0].page_content == "공학인증제도 안내 - 졸업이수 요건.md":
    EChain(docs, query)

else:
    print("1")

공학인증제도 안내 - 졸업이수 요건.md


Created a chunk of size 4060, which is longer than the specified 4000
Created a chunk of size 4396, which is longer than the specified 4000
Created a chunk of size 4375, which is longer than the specified 4000
Created a chunk of size 4278, which is longer than the specified 4000
Created a chunk of size 4799, which is longer than the specified 4000
Created a chunk of size 4772, which is longer than the specified 4000
Created a chunk of size 4534, which is longer than the specified 4000
Created a chunk of size 4803, which is longer than the specified 4000
Created a chunk of size 4492, which is longer than the specified 4000
Created a chunk of size 4212, which is longer than the specified 4000
Created a chunk of size 4847, which is longer than the specified 4000
Created a chunk of size 4655, which is longer than the specified 4000
Created a chunk of size 4566, which is longer than the specified 4000
Created a chunk of size 4692, which is longer than the specified 4000
Created a chunk of s

# 2024학번 입학자부터 적용 졸업요건

| **졸업요건** | **졸업이수학점 : 133학점(필수 포함)** |
| - | - |

## 졸업요건 세부사항

| 구분 | 학과 | 교양 | 전공 | 졸업요건 학점 | 최소 졸업요건 ‘공학필수’ |
| - | - | - | - | - | - |
| 전자 정보 공과 대학 | 전자공학과 | ① 학번에 따른 「교양 교과목 이수체계」참고<br>② MSC 24~30 학점<br>- 공학계열 단과 대학 학과별 「교양 및 MSC(수학, 기초 과학, 전산학) 교과 과정표」참고 | 전공 전필 포함 60학점 (설계 12학점 포함) | 공학설계입문, 캡스톤설계 |  |
|  | 전자통신공학과 |  | 전공 전필 포함 60학점 (설계 12학점 포함) | 공학설계입문, 예비캡스톤설계, 캡스톤설계 |  |
|  | 전자융합공학과 |  | 전공 전필 포함 60학점 (설계 12학점 포함) | 공학설계입문, 캡스톤설계1 |  |
|  | 전기공학과 |  | 전공 전필 포함 60학점 (설계 12학점 포함) | 공학설계입문, 캡스톤설계 |  |
|  | 전자재료공학과 |  | 전공 전필 포함 60학점 (설계 12학점 포함) | 공학설계입문, 캡스톤설계1, 캡스톤설계2 |  |
|  | 반도체시스템공학부 |  | 전공 전필 포함 60학점 (설계 12학점 포함) | 공학설계입문, 캡스톤종합설계1 |  |
| 인공 지능 융합 대학 | 컴퓨터정보공학부
